# MLflow DSPy Flavor

[mlflow.dspy.autolog()](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.dspy.html)

## Install MLflow

In [0]:
%pip install -U mlflow>=3.8.0 dspy>=3.0.4
dbutils.library.restartPython()

In [0]:
import mlflow

print(f"MLflow version: {mlflow.__version__}")

In [0]:
import dspy

print(f"DSPy version: {dspy.__version__}")

In [0]:
import mlflow

mlflow.get_tracking_uri()

# MLflow DSPy Autologging

[MLflow Tracing](https://mlflow.org/docs/latest/genai/tracing/integrations/) provides automatic tracing capability for DSPy.

Once enabled, nested traces are automatically logged to an active MLflow Experiment upon invocation of DSPy modules.

* Databricks' [Tracing DSPy](https://docs.databricks.com/aws/en/mlflow3/genai/tracing/integrations/dspy)
* MLflow's [Tracing DSPy](https://mlflow.org/docs/latest/genai/tracing/integrations/listing/dspy/)

## Enable Autologging

In [0]:
import mlflow

mlflow.dspy.autolog()

In [0]:
help(mlflow.dspy.autolog)

# 👨‍💻 Demo: DSPy Program

[Example Usage](https://mlflow.org/docs/latest/genai/tracing/integrations/listing/dspy/#example-usage)

In [0]:
import dspy

lm = dspy.LM(model="databricks/databricks-claude-sonnet-4-5")
dspy.configure(lm=lm)

In [0]:
class SummarizeSignature(dspy.Signature):
    """Given a passage, generate a summary."""

    passage: str = dspy.InputField(desc="a passage to summarize")
    summary: str = dspy.OutputField(desc="a one-line summary of the passage")

In [0]:
# Not really necessary
class Summarize(dspy.Module):
    def __init__(self):
        self.summarize = dspy.ChainOfThought(SummarizeSignature)

    def forward(self, passage: str):
        return self.summarize(passage=passage)

In [0]:
summarizer = Summarize()
summarizer(
    passage=(
        "MLflow Tracing is a feature that enhances LLM observability in your Generative AI (GenAI) applications "
        "by capturing detailed information about the execution of your application's services. Tracing provides "
        "a way to record the inputs, outputs, and metadata associated with each intermediate step of a request, "
        "enabling you to easily pinpoint the source of bugs and unexpected behaviors."
    )
)

# 👨‍💻 Demo: Tracing during Evaluation

[Tracing during Evaluation](https://mlflow.org/docs/latest/genai/tracing/integrations/listing/dspy/#tracing-during-evaluation)

In [0]:
import mlflow

mlflow.dspy.autolog(log_traces_from_eval=True)

In [0]:
import dspy

lm = dspy.LM(model="databricks/databricks-claude-sonnet-4-5")
dspy.configure(lm=lm)

In [0]:
class Counter(dspy.Signature):
    question: str = dspy.InputField()
    answer: str = dspy.OutputField(
        desc="Should only contain a single number as an answer"
    )

cot = dspy.ChainOfThought(Counter)

In [0]:
import dspy

eval_set = [
    dspy.Example(
        question="How many 'r's are in the word 'strawberry'?", answer="3"
    ).with_inputs("question"),
    dspy.Example(
        question="How many 'a's are in the word 'banana'?", answer="3"
    ).with_inputs("question"),
    dspy.Example(
        question="How many 'e's are in the word 'elephant'?", answer="2"
    ).with_inputs("question"),
]

In [0]:
from dspy.evaluate.metrics import answer_exact_match

# Evaluate the programs
with mlflow.start_run(run_name="CoT Evaluation"):
    evaluator = dspy.evaluate.Evaluate(
        devset=eval_set,
        # return_all_scores=True,
        # display_progress=True,
    )
    eval_results = evaluator(cot, metric=answer_exact_match)
    score = eval_results.score

    # Log the aggregated score
    mlflow.log_metric("exact_match", score)
    # Log the detailed evaluation results as a table
    mlflow.log_table(
        {
            "question": [example.question for example in eval_set],
            "answer": [example.answer for example in eval_set],
            "results": results,
        },
        artifact_file="eval_results.json",
    )

# mlflow.dspy.autolog

<br>

```python
autolog(
    log_traces: bool = True,
    log_traces_from_compile: bool = False,
    log_traces_from_eval: bool = True,
    log_compiles: bool = False,
    log_evals: bool = False,
    disable: bool = False,
    silent: bool = False,
)
```

In [0]:
import mlflow
help(mlflow.dspy.autolog)

## log_traces Argument

Enabled by default

`autolog(log_traces: bool = True)`

MLflow's "Patchers":
* `_patched_compile`
* `_patched_evaluate`

`safe_patch` the following methods:

| Module | Class | Method |
|-|-|-|
| `dspy.teleprompt` | `Teleprompter`<br>(incl. subclasses) | `compile` |
| `dspy.evaluate` | `Evaluate` | `__call__` |